In [1]:
import nltk
from sklearn.model_selection import train_test_split
nltk.download('vader_lexicon')
nltk.download('punkt')
from sklearn.linear_model import LogisticRegression
import pandas as pd

import code.auth_and_scrape as get_tweets
import code.preparation as prep
#import code.visualization as viz

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Twitter API Authentication

In [ ]:
get_tweets.authenticate(key, key_secret, access, access_secret)

Function to scrape a chosen amount of tweets with a chosen keyword and append to CSV file.

In [5]:
get_tweets.scrape_and_place()

# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900
# of Tweets added from this response:  900


In [50]:
df = pd.read_csv('data/data.csv')
df.head()

,id,created_at,language,favorites,retweets,text
0,1462933594754670600,2021-11-22 23:58:43+00:00,en,0,0,@DestinyTheGame all id like is to be at least ...
1,1462933098887266308,2021-11-22 23:56:45+00:00,en,0,0,@DestinyTheGame Can someone please explain to ...
2,1462932969086087176,2021-11-22 23:56:14+00:00,en,1,0,@techaftmidnight @GoogleStadia @DestinyTheGame...
3,1462932844418842632,2021-11-22 23:55:44+00:00,en,1,0,Going live in 5 minutes!! Let's go GOD SQUAD!!...
4,1462932760515985420,2021-11-22 23:55:24+00:00,en,0,0,"@A_dmg04 @DestinyTheGame Dear Eva, please brin..."


In [51]:
df.favorites.describe()

count    6300.000000
mean        9.520635
std        90.900910
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max      3879.000000
Name: favorites, dtype: float64

Cleaning of Tweets

In [52]:
df.text[1]

'@DestinyTheGame Can someone please explain to me why I (top 7-10%ish player) keep getting teamed with people in the… https://t.co/fREoFm7Emb'

This tweet has a ton of punctuation and it even includes a url. In order for our sentiment analyzer to read and understand this, we must remove the all of this and even lowercase every letter. Here A function has been create to do this.

In [54]:
df = prep.clean_tweets(df)

A function to create a Sentiment Analysis using TextBlob and SentimentIntensityAnalyzer.

In [56]:
df_probs = prep.create_sentiment(df)

Feature Engineering Positive Variable

In [58]:
df_pos_uncleaned = prep.create_pos_df(df, df_probs)

In [59]:
df_pos_cleaned = prep.clean_tweets(df_pos_uncleaned)

In [61]:
positives_and_count, pos_vec = prep.top_n_gram(df_pos_cleaned['clean_text'], (1,1), 300)

positives = [x[0] for x in positives_and_count]
top_10_pos = positives[:10]
positives_minus_top_10 = [x for x in positives if x not in top_10_pos]
print(positives_minus_top_10)

['destiny2', 'threadsoflight', 'game', 'play', 'congrats', 'thanks', 'bring', 'destinythegame', 'make', 'really', 'don', 'hope', 'great', 'time', 'nice', 'better', 'new', 'day', 'playing', 'looks', 'congratulations', 'yeah', 'bungie', 'know', 'happy', 'trials', 'help', 'awesome', 'think', 've', 'going', 'dawning', 'wish', 'year', 'fun', 'got', 'space', 'hunter', 'people', 'shader', 'look', 'best', 'd2', 'live', 'crow', 'grandma', 'free', 'let', 'need', 'amazing', 'today', 'twab', 'friends', 'll', 'guardian', 'lol', 'week', 'use', 'getting', 'god', 'cool', 'pvp', 'changes', 'work', 'come', 'double', 'amp', 'right', 'looking', 'titan', 'team', 'maybe', 'ability', 'actually', 'oh', 'win', 'vex', 'pretty', 'flawless', 'thememe', 'super', 'man', 'played', 'rewards']


Create Feature column.

In [62]:
features = [
    (prep.create_features(review, positives_minus_top_10))
    for review in df['clean_text']]

In [63]:
df_feat = pd.concat([df['clean_text'], pd.DataFrame(features,columns=['positive_word_count'])], axis=1)

Finally, train-test split

In [64]:
# Train-test split
X_train,X_test,y_train,y_test = train_test_split(df_feat, df_probs['sentiment'],random_state=1)
X_train

,clean_text,positive_word_count
4319,read my mind,0
518,oh god please no more cookie baking please,2
4584,ahh thanks so much ghaul lt 3,1
3768,these cool down changes in combination with da...,2
2990,they talking about why eris had to put her r...,1
...,...,...
905,getting back into sniping slowly des...,3
5192,congratulations hell yeah,2
3980,ab you have been having problems for years...,0
235,dear eva could you kindly ask ada 1 to sel...,0


In [66]:
feat, feat_vec_train = prep.top_n_gram(X_train['clean_text'], (1,1), 100)
feat, feat_vec_test = prep.top_n_gram_test(X_test['clean_text'], X_train['clean_text'], (1,1), 100)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

#Create X_train vectorized
X_train_vec = pd.concat([X_train['positive_word_count'], pd.DataFrame(feat_vec_train.todense())], axis=1)
X_test_vec = pd.concat([X_test['positive_word_count'], pd.DataFrame(feat_vec_test.todense())], axis=1)

Baseline model is a simple logistic regression that scored 78% accuracy

In [70]:
log_model = LogisticRegression()
log_model.fit(X_train_vec, y_train)
log_model.score(X_test_vec, y_test)

0.7873015873015873